In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
#import networkx as nx

In [ ]:
dataFolderPath = "/Users/adsriram98/Documents/PARK_LAB/PRS_calc/data"; #change this directory to wherever you are saving your data files to.

In [ ]:
'''
#Generate knockoff data using one of two methods: ISEE Omega and Cholesky_LU.
The code for generating ISEE Omega knockoff is implemented using R. Please make sure your computer has R installed.
'''
dataFolderPath = "/Users/adsriram98/Documents/PARK_LAB/PRS_calc/data";

from DL.knockoff.KnockoffGenerator import KnockoffGenerator;
generator = KnockoffGenerator();

DataFileName = "42p_82894samples_02-23-24_incidence_X.txt"

knockoffFilePath = generator.CholLuKnockoff(dataFolderPath, DataFileName,sep="\t");

print("The newly generated knockoff file is named as:");
print(knockoffFilePath);

In [ ]:
#for statistical knockoffs
XKnockoffData = pd.read_csv(knockoffFilePath,sep="\t");
XKnockoffData

In [ ]:
#for PRS Knockoffs
XKnockoffData = pd.read_csv("/Users/adsriram98/Documents/PARK_LAB/PRS_calc/data/02-22-24-PRSknockoffs-incidence.csv", sep=",")
XKnockoffData

In [ ]:
''''''
# After generating the knockoff data, run DNN

#dataFolderPath = "data";
dataFolderPath = "/Users/adsriram98/Documents/PARK_LAB/PRS_calc/data"


YDataFileName = '42p_82894samples_02-23-24_incidence_Y.txt';
Ydata = pd.read_csv(dataFolderPath+os.path.sep+YDataFileName,sep="\t");

XKValues = XKnockoffData.values;
YValues = Ydata.values;
    
pNum = int(XKValues.shape[1] / 2);
n = XKValues.shape[0];
print(XKValues.shape);
print(YValues.shape);
print(pNum);
    
XOrigin = XKValues[:, 0:pNum];
knockoff = XKValues[:, pNum:];

X3DTrain = np.zeros((n, pNum, 2));
X3DTrain[:, :, 0] = XOrigin;
X3DTrain[:, :, 1] = knockoff;
labelTrain = YValues;
    
coeff = 0.05 * np.sqrt(2.0 * np.log(pNum) / n);

nOutputs = Ydata.shape[1];

#Save the DNN output to the following directory.
resultDir = dataFolderPath+os.path.sep+'02_23_24_STATknockoffs_sepsis_incidence_UKB_DNN_result/'; #change this directory to the directory you are using
if not os.path.exists(resultDir):
    os.makedirs(resultDir);
    
from DL.DNN.DNN import DNN;
dnn = DNN();
model = dnn.build_DNN(pNum, nOutputs, coeff);
callback = DNN.Job_finish_Callback(resultDir,pNum);
dnn.train_DNN(model, X3DTrain, labelTrain,callback);

In [ ]:
# Apply FDR control to DNN result
from DL.FDR.FDR_control import FDR_control
control = FDR_control()

XDataFileName = "42p_82894samples_02-23-24_incidence_X.txt"
selected_features = control.controlFilter(dataFolderPath + os.path.sep + XDataFileName, resultDir, offset=1, q=0.00)

# Save the selected associations with statistic and threshold
selected_associations = []
for feature, statistic, threshold in selected_features:
    selected_associations.append({
        "Feature1": feature,
        "Feature2": "Y",
        "Statistic": statistic,
        "Threshold": threshold
    })

pd.DataFrame(selected_associations).to_csv(dataFolderPath + os.path.sep + "02_23_24_STATknockoffs_SEPSISincidence_UKB_DNN_selected_associations.csv"); #change this directory to the directory you are using

